Librerias

In [22]:
import pandas as pd
import numpy as np

En este notebook se reestructura las variables del dataset original para facilitar el análisis posterior. Para ello, se revisa con los doctores el manejo de fechas y dias. Adicionalmente, como los datos originales vienen codificados, se utiliza el codebook para decodificar las categorías. De está forma, se crea una nueva hoja que en vez de los números de las categorías contenga las categorías en texto. Esta nueva hoja se guarda como un nuevo archivo excel para subir al sharepoint que se usará en el Power BI y Power Apps.

Se carga el codebook para entender las variables.

In [23]:
df_codebook = pd.read_json('../data/Codebook.json')
df_codebook

,Variable,Descripción,Tipo de Variable,Valores
0,Código anonimizado,código anonimizado para tratamiento de datos d...,Texto,{}
1,#Paciente_Tx,número de paciente,Númerica Discreta,{}
2,Año_Tx,año del trasplante,Númerica Discreta,{}
3,Fecha_Tx,tiempo transcurrido desde el trasplante hepático,Fecha,{}
4,Etiología_#1,enfermedad que lo llevó a la enfermedad hepática,Categórica,"{'0': 'NINGUNA', '1': 'HCV', '2': 'HBV', '3': ..."
...,...,...,...,...
56,Vivo_Hoy,define si está vivo hoy,Categórica,"{'0': 'NO', '1': 'SI'}"
57,Fecha_Control/Muerte,fecha de control/muerte,Fecha,{}
58,SOBREVIDA_DIAS,sobrevida en días post trasplante,Númerica Discreta,{}
59,SOBREVIDA_MESES,sobrevida en meses post trasplante,Númerica Continua,{}


Se carga la base de datos limpia.

In [24]:
data = pd.read_excel('../data/Base_infecciones_POPTH_limpia.xlsx')

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 61 columns):
 #   Column                                      Non-Null Count  Dtype         
---  ------                                      --------------  -----         
 0   Código anonimizado                          557 non-null    object        
 1   #Paciente_Tx                                557 non-null    int64         
 2   Año_Tx                                      557 non-null    int64         
 3   Fecha_Tx                                    557 non-null    datetime64[ns]
 4   Etiología_#1                                557 non-null    int64         
 5   Etiología_#2                                557 non-null    int64         
 6   Edad                                        557 non-null    int64         
 7   Sexo                                        557 non-null    int64         
 8   Child_Pugh_Score_Cat                        158 non-null    object        
 9   Child_Pugh

Creamos un dataframe nuevo donde se guardará la estructuración nueva de las variables y la decodificación de las categorías.

In [26]:
data_new = data.copy()

# Fechas

Una ruta típica de un paciente trasplantado es la siguiente:

Ingreso hospital ──(preoperatorio)──► Cirugía Tx ─► UCI ─► Piso (hospitalización común) ─► Alta

Para modelar adecuadamente los días de estancia en cada fase, es necesario contar con las siguientes fechas clave:
- Fecha_Ingreso_Hospitalario (No existe en el dataset, la añadimos)
- Fecha_Trasplante (ya existe en el dataset)
- Fecha_Egreso_Hospitalaria (ya existe en el dataset)

Y casillas de apoyo:
- "UCI_al_Momento_Tx" (Sí/No)
- ¿Estuvo en UCI 6 meses pretrasplante? Sí/No → si Sí, “Número total de días en UCI pre-Tx”.
- ¿Estuvo en UCI postrasplante? Sí/No → si Sí, “Número total de días en UCI post-Tx”.
- ¿Estuvo en piso postrasplante? Sí/No → si Sí, “Número total de días en piso post-Tx”.

No se preguntan fechas específicas de ingreso y alta en UCI o piso ya que los episodios de UCI y piso pueden ser múltiples y variados en el tiempo, pero con las fechas clave y las casillas de apoyo se puede calcular la duración total en cada fase.


Con esto, las variables de días que se pueden derivar son las siguientes:
- Dias_Estancia_Hospitalaria: número de días transcurridos entre el ingreso hospitalario y el alta, incluyendo el periodo pre y postrasplante.
  - variable calculada como: Fecha_Egreso_Hospitalaria - Fecha_Ingreso_Hospitalario
- Días_Totales_Intrahosspitalarios_Pre_Tx: número de días desde la fecha de ingreso hospitalario hasta la fecha del trasplante.
  - variable calculada como: Fecha_Trasplante - Fecha_Ingreso_Hospitalario
- Días_En_UCI_Pre_Tx: días de estancia en UCI previos al trasplante en una ventana de 6 meses previos al trasplante.
- Días_Totales_Intrahospitalarios_Post_Tx: número de días desde la fecha del trasplante hasta el alta hospitalaria.
  - variable calculada como: Fecha_Egreso_Hospitalaria - Fecha_Trasplante
- Días_En_UCI_Post_Tx: días de estancia en UCI posteriores al trasplante.
- Días_En_Hospitalización_Piso_Post_Tx: días de hospitalización en piso posteriores al trasplante.

In [27]:
# Actualmente las variables de días que se encuentran son las siguientes:
days_vars = [
    "Antecedente_UCI_6_meses_PreOP",
    "Dias_Estancia_Hospitalaria",
    "Días_Hospitalización_UCI",
    "Días_Totales_Intrahospitalarios",
    "Días_En_Hospitalización_Piso",
    "Días_En_UCI_POP",
]
data[days_vars]

,Antecedente_UCI_6_meses_PreOP,Dias_Estancia_Hospitalaria,Días_Hospitalización_UCI,Días_Totales_Intrahospitalarios,Días_En_Hospitalización_Piso,Días_En_UCI_POP
0,0.0,10,0.0,10.0,7.0,3.0
1,0.0,1,0.0,1.0,0.0,1.0
2,0.0,11,0.0,13.0,6.0,7.0
3,0.0,9,0.0,9.0,4.0,5.0
4,0.0,8,0.0,7.0,4.0,3.0
...,...,...,...,...,...,...
552,0.0,0,0.0,NaN,6.0,0.0
553,1.0,0,0.0,NaN,NaN,NaN
554,0.0,0,0.0,NaN,4.0,0.0
555,0.0,0,0.0,NaN,9.0,0.0


In [28]:
# Vamos a renombrar las variables de días para que queden más claras en cuanto a si son pre o post trasplante
dict_rename_days = {
    "Antecedente_UCI_6_meses_PreOP": "UCI_6_meses_Pre_Tx",
    "Días_Hospitalización_UCI": "Días_En_UCI_Pre_Tx",
    "Días_Totales_Intrahospitalarios": "Días_Totales_Intrahospitalarios_Post_Tx",
    "Días_En_Hospitalización_Piso": "Días_En_Hospitalización_Piso_Post_Tx",
    "Días_En_UCI_POP": "Días_En_UCI_Post_Tx",
}
data_new = data_new.rename(columns=dict_rename_days)

# renombramos también en el codebook, conservando las variables que no son de días
df_codebook["Variable"] = df_codebook["Variable"].map(lambda x: dict_rename_days.get(x, x))

In [29]:

# vamos a añadir la fecha de ingreso hospitalario
data_new['Fecha_Ingreso_Hospitalario'] = pd.NaT
# intentar estimar con la fecha de egreso hospitalario y los días de estancia hospitalaria
data_new.loc[
    data_new['Fecha_Egreso_Hospitalario'].notna() & data_new['Dias_Estancia_Hospitalaria'].notna(),
    'Fecha_Ingreso_Hospitalario' ] = data_new['Fecha_Egreso_Hospitalario'] - pd.to_timedelta(data_new['Dias_Estancia_Hospitalaria'], unit='D')

# np.nan if fecha ingeso es mayor a fecha trasplante
data_new.loc[
    data_new['Fecha_Ingreso_Hospitalario'] > data_new['Fecha_Tx'], 'Fecha_Ingreso_Hospitalario'] = pd.NaT


# Vamos a añadir las variables de días pre trasplante que faltan
data_new['Días_Totales_Intrahospitalarios_Pre_Tx'] = np.nan
# intentamos estimar con la fecha de ingreso hospitalario y la fecha de trasplante
data_new.loc[
    data_new['Fecha_Ingreso_Hospitalario'].notna() & data_new['Fecha_Tx'].notna(),
    'Días_Totales_Intrahospitalarios_Pre_Tx'] = (data_new['Fecha_Tx'] - data_new['Fecha_Ingreso_Hospitalario']).dt.days

 Días_Totales_Intrahospitalarios_Post_Tx

In [30]:
# vamos a intentar estimar los días de estancia en hospitalización post trasplante
dates_validation = data_new["Fecha_Egreso_Hospitalario"] - data_new["Fecha_Tx"]
calculated_values = data_new["Días_En_UCI_Post_Tx"] + data_new["Días_En_Hospitalización_Piso_Post_Tx"]
actual_values = data_new["Días_Totales_Intrahospitalarios_Post_Tx"]
check1 = calculated_values == actual_values

# donde dias totales de intrahospitalarios post tx es nulo y calculado no es nulo, lo llenamos
data_new.loc[actual_values.isna() & calculated_values.notna(), "Días_Totales_Intrahospitalarios_Post_Tx"] = calculated_values

# donde no hay fecha de egreso hospitalario, la estimamos con la suma de días en UCI y días en piso
data_new.loc[data_new["Fecha_Egreso_Hospitalario"].isna(), "Fecha_Egreso_Hospitalario"] = data_new["Fecha_Tx"] + pd.to_timedelta(calculated_values, unit='D')


# donde la fecha de egreso sea menor a la fecha de trasplante, la ponemos como nulo
data_new.loc[
    data_new["Fecha_Egreso_Hospitalario"] < data_new["Fecha_Tx"], "Fecha_Egreso_Hospitalario"] = pd.NaT

df_temp = pd.DataFrame({
    "dates validation": dates_validation.dt.days,
    "calculated_days": calculated_values,
    "actual_days": actual_values,
    "check 1": check1
})
df_temp

,dates validation,calculated_days,actual_days,check 1
0,NaN,10.0,10.0,True
1,NaN,1.0,1.0,True
2,NaN,13.0,13.0,True
3,NaN,9.0,9.0,True
4,NaN,7.0,7.0,True
...,...,...,...,...
552,6.0,6.0,6.0,False
553,NaN,NaN,NaN,False
554,4.0,4.0,4.0,False
555,9.0,9.0,9.0,False


Adicionlmente, se preguntará la fecha de control/muerte para calcular la sobrevida en días, meses y años desde la fecha del trasplante.
Está variable ya existe en el dataset original.
Se conservará tal cual como está.

In [31]:
orden = ["Código anonimizado", "#Paciente_Tx", "Etiología_#1", "Etiología_#2", "Edad", "Sexo",	
         "Child_Pugh_Score_Cat", "Child_Pugh_Score", "Meld_Score","Diabetes_Mellitus","Tabaquismo","Alcoholismo",
        "Hipertensión_Arterial", "Peso_previo_Cx",	"Talla", "IMC", "Año_Tx", "Fecha_Tx", 'Fecha_Ingreso_Hospitalario', 'Días_Totales_Intrahospitalarios_Pre_Tx', "UCI_6_meses_Pre_Tx",  "Días_En_UCI_Pre_Tx", "UCI_al_Momento_Tx",	
        "Tiempo_Isquemia_Fria",	"Tiempo_Isquemia_caliente",	"Tiempo_Cx", "Tipo_Reconstrucción_Biliar",	
        "Infección_Al_Momento_Tx",	"Localización de la infección",	"Germen/Microorganismo aislado",	
        "Antibiotico_previo_al_Tx",	"Días_Tratamiento_Antibiótico_Previo_A_Tx#1",	
        "Antibiótico_Profiláctico_VS_Terapéutico",	"Antibiótico_1__Tx#1",	"Antibiótico_2_Tx#1",	
        "Antibiótico_3_Tx#1", "Antifúngico_Tx#1",	"Tiempo_De_Dosis_Hasta_Tx#1",	
        "Nutrición_Enteral","Días_Nutrición_Enteral",	"Complicaciones_Técnicas",	"Dias_Estancia_Hospitalaria",	
        "Días_En_UCI_Post_Tx", "Días_En_Hospitalización_Piso_Post_Tx",	"Días_Totales_Intrahospitalarios_Post_Tx",	
        "Inmunosupresión_con_Anticuerpos",	"Inmunosupresor_1_Postx",	"Inmunosupresor_2_PostTx",	
        "Inmunosupresor_1_1mesPostTx",	"Inmunosupresor_2_PostTx_1mesPx",	"Inmunosupresor_1_6mesesPostx",	
        "Inmunosupresor_2_6mesesPostTx", "Fecha_Egreso_UCI",	"Fecha_Egreso_Hospitalario",	
        "Requerimiento_de_diálisis","Trasfusión_GRE_hasta_1m_POP",	
        "Reintervención_Quirúrgica_hasta_1m_POP",	"Retrasplante",	
        "Vivo_Hoy",	"Fecha_Control/Muerte",	"SOBREVIDA_DIAS",	"SOBREVIDA_MESES",	"SOBREVIDA_AÑOS"]

In [32]:
data_new = data_new[orden]

In [33]:
# Se elimina está variable porque los valores se encuentran vacíos en su mayoria y no coincide con los días en UCI post trasplante
# por lo que se decide eliminarla
data_new = data_new.drop(columns=['Fecha_Egreso_UCI'])

Con data new se crea el excel que se subirá para crear la lista de sharepoint:

En el se configurará para calcular las siguientes variables:
- Dias_Estancia_Hospitalaria: Fecha_Egreso_Hospitalario - Fecha_Ingreso_Hospitalario
- Días_Totales_Intrahospitalarios_Pre_Tx: Fecha_Tx - Fecha_Ingreso_Hospitalaria
- Días_Totales_Intrahospitalarios_Post_Tx: Fecha_Egreso_Hospitalaria - Fecha_Tx
- Sobrevida_Dias: Fecha_Control/Muerte - Fecha_Tx
- Sobrevida_Meses: Sobrevida_Dias / 30
- Sobrevida_Años: Sobrevida_Dias / 365

El resto de variables se preguntarán directamente en el formulario de powerapps.

# Decodificación de categorías

Llenamos las variables númericas con -1.

In [34]:
numeric_vars = data_new.select_dtypes(include=['int64', 'float64']).columns
data_new[numeric_vars] = data_new[numeric_vars].fillna(-1)

Extraemos las variables categóricas y creamos un diccionario para decodificarlas usando el codebook.

In [35]:
df_categoricas = df_codebook[df_codebook['Tipo de Variable']=='Categórica']
mapping_dict = dict(zip(df_categoricas['Variable'], df_categoricas['Valores']))
mapping_dict

{'Etiología_#1': {'0': 'NINGUNA',
  '1': 'HCV',
  '2': 'HBV',
  '3': 'NASH',
  '4': 'ALCOHÓLICA',
  '5': 'HEMOCROMATOSIS',
  '6': 'DEF ALFA 1 ANTITRIPSINA',
  '7': 'WILSON',
  '8': 'HEPATITIS AUTOINMNE',
  '9': 'COLANGITIS BILIAR PRIMARIA',
  '10': 'COLANGITIS ESCLEROSANTE PRIMARIA',
  '11': 'OVERLAP (CBP + AIH)',
  '12': 'CRIPTOGENICA',
  '13': 'FIBROSIS HEPATICA CONGENITA',
  '14': 'ATRESIA VIAS BILIARES',
  '15': 'COLANGIOCARCINOMA',
  '16': 'TUMOR MIOFIBROBLASTICO',
  '17': 'HEPATOBLASTOMA',
  '18': 'ADENOMA GIGANTE',
  '19': 'HIPERPLASIA NODULAR PROLIFERATIVA',
  '20': 'ENFERMEDAD POLIQUISTICA',
  '21': 'BUDDCHIARI',
  '22': 'CIRROSIS BILIAR SECUNDARIA',
  '23': 'FHF',
  '24': 'HELLP',
  '25': 'CIRROSIS DEL INJERTO',
  '26': 'CIRROSIS DEL INJERTO',
  '27': 'CIRROSIS HEPATICA POR COLANGIOPATÍA AUTOINMUNE'},
 'Etiología_#2': {'0': 'NINGUNA',
  '1': 'HCV',
  '2': 'HBV',
  '3': 'NASH',
  '4': 'ALCOHÓLICA',
  '5': 'HEMOCROMATOSIS',
  '6': 'DEF ALFA 1 ANTITRIPSINA',
  '7': 'WILSON',
  '

Mapeamos los codigos a las categorías correspondientes y llenamos con "DESCONOCIDO" los valores que no se encuentren en el diccionario.

In [36]:
def map_values(column):
    if column.name in mapping_dict:
        mapping = mapping_dict.get(column.name)
        if pd.isna(mapping):
            return column
        column_mapped = column.map(lambda x: mapping.get(str(int(x)), x) if pd.notna(x) and str(int(x)) in mapping else x).replace(-1.0, "DESCONOCIDO")
        return column_mapped
    return column
data_mapped = data_new.apply(map_values)
data_mapped

,Código anonimizado,#Paciente_Tx,Etiología_#1,Etiología_#2,Edad,Sexo,Child_Pugh_Score_Cat,Child_Pugh_Score,Meld_Score,Diabetes_Mellitus,...,Fecha_Egreso_Hospitalario,Requerimiento_de_diálisis,Trasfusión_GRE_hasta_1m_POP,Reintervención_Quirúrgica_hasta_1m_POP,Retrasplante,Vivo_Hoy,Fecha_Control/Muerte,SOBREVIDA_DIAS,SOBREVIDA_MESES,SOBREVIDA_AÑOS
0,P001,1,HCV,NINGUNA,62,M,NaN,-1.0,18.0,NO,...,2009-01-13,NO,NO,SI,NO,NO,2009-10-05,275,9.17,0.76
1,P002,2,NASH,NINGUNA,62,M,NaN,-1.0,17.0,NO,...,2009-01-11,NO,NO,SI,NO,NO,2009-01-11,1,0.03,0.00
2,P003,3,HEPATITIS AUTOINMNE,NINGUNA,56,F,NaN,-1.0,15.0,NO,...,2009-02-16,NO,NO,NO,NO,SI,2023-12-31,5444,181.47,15.12
3,P004,4,NASH,NINGUNA,58,M,NaN,-1.0,9.0,NO,...,2009-02-14,NO,NO,NO,NO,SI,2023-12-31,5442,181.40,15.12
4,P005,5,ALCOHÓLICA,NINGUNA,62,F,NaN,-1.0,22.0,SI,...,2009-02-18,NO,NO,NO,NO,SI,2023-12-31,4724,157.47,13.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,P553,553,CRIPTOGENICA,NINGUNA,22,M,C,11.0,22.0,NO,...,2024-09-27,NO,NO,NO,NO,SI,2024-09-21,0,0.00,0.00
553,P554,554,HEPATITIS AUTOINMNE,NINGUNA,33,F,C,10.0,21.0,NO,...,NaT,SI,SI,DESCONOCIDO,DESCONOCIDO,NO,2025-03-31,174,5.00,0.00
554,P555,555,NASH,ALCOHÓLICA,55,M,B,9.0,20.0,SI,...,2024-10-12,NO,NO,NO,NO,SI,2025-05-07,203,6.00,0.00
555,P556,556,CIRROSIS HEPATICA POR COLANGIOPATÍA AUTOINMUNE,NINGUNA,54,F,B,9.0,14.0,NO,...,2024-10-25,NO,NO,NO,NO,SI,2025-05-13,196,6.00,0.00


Guardamos el nuevo dataset como un archivo excel para subir al sharepoint.

In [37]:
data_mapped.to_excel('../data/Base_infecciones_POPTH_nuevo_modelado.xlsx', index=False)